In [1]:
import gc
import numpy as np 
import pandas as pd
import optuna
from sklearn.metrics import log_loss
import warnings

warnings.filterwarnings('ignore')



In [2]:
%%time
h5File = "temp_data/h5/selectkbest_data_with_last.h5";
key_train = 'bestkselect_01/train';
key_valid = 'bestkselect_01/valid';
key_test = 'bestkselect_01/test';

train = pd.read_hdf(h5File, key_train);
valid = pd.read_hdf(h5File, key_valid);
test = pd.read_hdf(h5File, key_test);


CPU times: user 11.3 s, sys: 17.1 s, total: 28.4 s
Wall time: 36.1 s


In [3]:
train

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,...,D_132_last,D_132_ldiff,D_133_last,D_136_last,D_139_last,D_140_last,D_141_last,D_143_last,D_145_last,S_2_min_m
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0.868580,0.960384,0.933824,0.091505,0.010704,0.001930,0.021655,0.012007,1.000242,1.009672,...,0.0,0.0,0.006210,0.000000,0.007186,0.004234,0.005086,0.005810,0.008533,3
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0.861109,0.929122,0.899820,0.567403,0.215205,0.006711,0.109644,0.025654,0.819772,1.008534,...,0.0,0.0,0.002996,0.000000,0.002980,0.007479,0.007870,0.003284,0.008514,3
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0.797670,0.904482,0.878454,0.009704,0.004181,0.001472,0.009997,0.004386,0.810796,0.819987,...,0.0,0.0,0.009881,0.000000,0.007383,0.006623,0.000964,0.002202,0.003444,3
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0.567442,0.623392,0.598969,0.268476,0.048862,0.005910,0.279991,0.059876,0.812053,1.009999,...,0.0,0.0,0.001789,0.000000,0.002704,0.006184,0.001899,0.008183,0.002983,3
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0.805045,0.940382,0.891679,0.008680,0.004644,0.000776,0.009806,0.005941,0.810670,0.819947,...,0.0,0.0,0.005045,0.000000,0.002974,0.004162,0.005764,0.008154,0.000905,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f,0.036622,0.428700,0.305832,0.765157,0.145896,0.053321,0.204184,0.145427,0.051535,0.814995,...,0.0,0.0,0.004369,0.000000,0.000320,0.004045,0.002236,0.004224,0.002413,7
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311,0.164876,0.384758,0.312491,0.766661,0.255978,0.451088,1.322943,0.693505,0.008950,0.038503,...,0.0,0.0,0.000537,0.000000,0.005447,0.002212,0.008570,0.009164,0.007635,3
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956,0.520425,0.677117,0.582655,1.036184,0.308401,0.111283,0.498702,0.325557,0.050609,0.433234,...,0.0,0.0,0.005766,0.000000,0.000154,0.007504,0.007130,0.006720,0.003733,3


In [4]:
valid

customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    0.0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    0.0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc    0.0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed    0.0
                                                                   ... 
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f    1.0
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311    1.0
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956    0.0
e0dd131940680fb03f52d808f3c1308abf4b057188fc0540614d2a543ab8a1ea    0.0
e0dd17ec1fae28e858808935b974ce142d53238d657018a1a2f129335e62d388    1.0
Name: target, Length: 458913, dtype: float64

In [5]:
test

,P_2_min,P_2_max,P_2_avg,D_39_max,D_39_avg,B_1_min,B_1_max,B_1_avg,B_2_min,B_2_max,...,D_132_last,D_132_ldiff,D_133_last,D_136_last,D_139_last,D_140_last,D_141_last,D_143_last,D_145_last,S_2_min_m
customer_ID,,,,,,,,,,,,,,,,,,,,,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.568930,0.631315,0.601387,0.238794,0.069758,0.007959,0.021672,0.013925,0.810456,1.009347,...,0.000000,0.000000,0.006273,0.0,0.005912,0.001250,0.006543,0.009160,0.003219,2
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.794469,0.913501,0.862166,0.505795,0.154252,0.014187,0.276014,0.052342,1.000782,1.009245,...,0.000000,0.000000,0.002767,0.0,0.004344,0.000866,0.009120,0.002196,0.007780,4
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.673112,0.835114,0.748955,0.679547,0.181814,0.001483,0.039697,0.012762,0.810072,1.009582,...,0.000000,0.000000,0.002045,0.0,1.001246,0.008894,0.896224,1.009391,0.092041,10
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.428457,0.514222,0.474728,0.683640,0.470512,0.149511,0.309129,0.284608,0.032696,0.205678,...,0.000000,0.000000,0.009377,0.0,1.008246,0.003753,0.919774,1.008100,0.183020,4
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.254478,0.425764,0.324100,0.768016,0.353050,0.357828,0.563603,0.456779,0.020167,0.106739,...,0.000000,0.000000,0.007940,0.0,0.006623,0.001140,0.009527,0.009408,0.000525,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.597667,0.646915,0.631613,0.009711,0.003932,0.001874,0.011309,0.007622,0.810936,0.819394,...,0.000000,0.000000,0.009844,0.0,0.003015,0.006851,0.009307,0.009393,0.005294,4
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.471303,0.634783,0.547049,0.417259,0.056910,0.015243,0.084163,0.049328,0.082507,1.009882,...,0.000000,0.000000,0.001985,0.0,0.007684,0.003375,0.009780,0.008109,0.009981,10
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.195671,0.316648,0.239070,1.119398,0.232943,0.013811,0.051597,0.031747,0.171140,1.009973,...,0.145253,0.053708,0.274885,0.0,0.002304,0.001640,0.002371,0.008154,0.002120,4


In [6]:
X_train = train
y_train = valid
X_test = test

In [22]:
# X_train = train[:1000]
# y_train = valid[:1000]
# X_test = test[:1000]

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)
print(X_train.head())
print(X_valid.head())
print(y_train.head())
print(y_valid.head())

                                                     P_2_min   P_2_max  \
customer_ID                                                              
0029d45c7fd5b96eaeaef3dc73d88a9aa0f9bf22c010ac7...  0.800555  0.821940   
0008c2f297e1b00bf567c0d2c25f3e3b356f9a3088d2bf4...  0.290994  0.565513   
00676bbba68dc87ccb2485d6c2ced9dd60be9912974ee3c...  0.937742  0.984307   
006d23fc91d62980c6ba065cc68d9489b73553dc48df77a...  0.974451  1.009723   
008cd7f666bf865907cccfd40db106451354c3c6cd4020b...  0.246159  0.639961   

                                                     P_2_avg  D_39_max  \
customer_ID                                                              
0029d45c7fd5b96eaeaef3dc73d88a9aa0f9bf22c010ac7...  0.810323  0.009857   
0008c2f297e1b00bf567c0d2c25f3e3b356f9a3088d2bf4...  0.494971  0.353181   
00676bbba68dc87ccb2485d6c2ced9dd60be9912974ee3c...  0.964542  0.294949   
006d23fc91d62980c6ba065cc68d9489b73553dc48df77a...  1.000254  0.007407   
008cd7f666bf865907cccfd40db106451354c

In [17]:
from xgboost import XGBClassifier

def objective(trial):

    print("--- train ---")
    model = XGBClassifier(
        learning_rate=0.1,
        random_state=55,
        early_stopping_rounds=10,
        n_estimators=1000,
        booster=trial.suggest_categorical('booster', ['gbtree', 'dart']),
        max_depth=trial.suggest_int('max_depth', 4, 10),
        min_child_weight=trial.suggest_uniform('min_child_weight', 0.1, 10),
        gamma=trial.suggest_uniform('gamma', 0.01, 10),
        reg_alpha=trial.suggest_uniform('reg_alpha', 0.1, 2.0),
        reg_lambda=trial.suggest_uniform('reg_lambda', 0.1, 2.0),
    )
    eval_set = [(X_valid, y_valid)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=True)
    
    print("--- oof_train ---")
    y_pred_valid = model.predict(X_valid)
    print("--- score ---")
    score = log_loss(y_valid, y_pred_valid)
    print('score:', score)
    
    return score

In [18]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=30)

[I 2022-08-21 23:09:28,459] A new study created in memory with name: no-name-131f69ee-bff8-4658-a880-2752e933d2a9


--- train ---
[0]	validation_0-logloss:0.63844
[1]	validation_0-logloss:0.59327
[2]	validation_0-logloss:0.55481
[3]	validation_0-logloss:0.52102
[4]	validation_0-logloss:0.48891
[5]	validation_0-logloss:0.46490
[6]	validation_0-logloss:0.44200
[7]	validation_0-logloss:0.42221
[8]	validation_0-logloss:0.40376
[9]	validation_0-logloss:0.38706
[10]	validation_0-logloss:0.37417
[11]	validation_0-logloss:0.36140
[12]	validation_0-logloss:0.35215
[13]	validation_0-logloss:0.34348
[14]	validation_0-logloss:0.33461
[15]	validation_0-logloss:0.32762
[16]	validation_0-logloss:0.32042
[17]	validation_0-logloss:0.31614
[18]	validation_0-logloss:0.31028
[19]	validation_0-logloss:0.30625
[20]	validation_0-logloss:0.30207
[21]	validation_0-logloss:0.29644
[22]	validation_0-logloss:0.29196
[23]	validation_0-logloss:0.28790
[24]	validation_0-logloss:0.28522
[25]	validation_0-logloss:0.28249
[26]	validation_0-logloss:0.28052
[27]	validation_0-logloss:0.27806
[28]	validation_0-logloss:0.27423
[29]	valid

[I 2022-08-21 23:09:30,611] Trial 0 finished with value: 3.684168132687681 and parameters: {'booster': 'dart', 'max_depth': 8, 'min_child_weight': 5.494343511669279, 'gamma': 4.242311445395658, 'reg_alpha': 1.3271988148266467, 'reg_lambda': 0.9314157013991157}. Best is trial 0 with value: 3.684168132687681.


--- score ---
score: 3.684168132687681
--- train ---
[0]	validation_0-logloss:0.63780
[1]	validation_0-logloss:0.59272
[2]	validation_0-logloss:0.55446
[3]	validation_0-logloss:0.52269
[4]	validation_0-logloss:0.49203
[5]	validation_0-logloss:0.46774
[6]	validation_0-logloss:0.44666
[7]	validation_0-logloss:0.42778
[8]	validation_0-logloss:0.41106
[9]	validation_0-logloss:0.39644
[10]	validation_0-logloss:0.38240
[11]	validation_0-logloss:0.37124
[12]	validation_0-logloss:0.35985
[13]	validation_0-logloss:0.35127
[14]	validation_0-logloss:0.34285
[15]	validation_0-logloss:0.33805
[16]	validation_0-logloss:0.33305
[17]	validation_0-logloss:0.32654
[18]	validation_0-logloss:0.32085
[19]	validation_0-logloss:0.31461
[20]	validation_0-logloss:0.31076
[21]	validation_0-logloss:0.30694
[22]	validation_0-logloss:0.30306
[23]	validation_0-logloss:0.29863
[24]	validation_0-logloss:0.29579
[25]	validation_0-logloss:0.29214
[26]	validation_0-logloss:0.29000
[27]	validation_0-logloss:0.28840
[28]	

[I 2022-08-21 23:09:32,679] Trial 1 finished with value: 3.684168132687681 and parameters: {'booster': 'dart', 'max_depth': 6, 'min_child_weight': 7.9380778770183795, 'gamma': 5.293660248331515, 'reg_alpha': 1.1792846660784715, 'reg_lambda': 1.8586336127560559}. Best is trial 0 with value: 3.684168132687681.


--- score ---
score: 3.684168132687681


In [20]:
study.best_params

{'booster': 'dart',
 'max_depth': 8,
 'min_child_weight': 5.494343511669279,
 'gamma': 4.242311445395658,
 'reg_alpha': 1.3271988148266467,
 'reg_lambda': 0.9314157013991157}

In [21]:
path = f'./output/optuna/502_param-tune_xboost.txt'
s = str(study.best_params)
with open(path, mode='w') as f:
    f.write(s)
